## Демонстрация лекционного материала

### Автоматическое управление памятью

**Reference counting**

Каждый объект имеет счетчик ссылок. Когда ссылка на объект удаляется, счетчик уменьшается. Когда счетчик достигает 0, объект удаляется.

Узнать количество ссылок на объект можно с помощью функции `sys.getrefcount`.

Важно отметить, что при вызове `sys.getrefcount` создаётся временная ссылка, поэтому количество ссылок на объект будет на 1 больше, чем фактическое количество ссылок.

In [ ]:
import sys

def demonstrate_reference_counting():
    """Демонстрирует работу reference counting."""
    
    # Создаем объект
    obj = [1, 2, 3]
    print(f"Счетчик ссылок: {sys.getrefcount(obj)}")
    
    # Создаем еще одну ссылку
    obj2 = obj
    print(f"После создания ссылки: {sys.getrefcount(obj)}")
    
    # Удаляем ссылку
    del obj2
    print(f"После удаления ссылки: {sys.getrefcount(obj)}")
    
    # Удаляем оригинальную ссылку
    del obj
    # Объект автоматически удаляется

demonstrate_reference_counting()

**Garbage collection**

Иногда в коде возникают так называемые циклические ссылки. Например, если объект A содержит ссылку на объект B, а объект B содержит ссылку на объект A, то эти объекты не будут удалены через reference counting, потому что счетчик ссылок никогда не будет равен 0.

Для решения этой проблемы используется garbage collection. Сборщик мусора периодически проверяет объекты и удаляет те, которые не используются.

Для отслеживания и управления процессом garbage collection используется модуль `gc`. Например, чтобы запустить сборщик мусора, можно использовать функцию `gc.collect()`.

In [ ]:
import gc

class Node:
    def __init__(self, name: str):
        self.name = name
        self.ref = None
        
    def __del__(self):
        print(f"Объект {self.name} удален")

def demonstrate_garbage_collection():
    """Демонстрирует работу сборщика мусора Python."""
    print("=== Демонстрация циклических ссылок ===")
    
    # Отключаем автоматический сборщик мусора
    gc.disable()
    
    # Создаем циклические ссылки
    node_a = Node("A")
    node_b = Node("B")
    
    # Создаем циклическую ссылку
    node_a.ref = node_b
    node_b.ref = node_a
    
    print(f"Объектов в памяти до удаления ссылок: {len([o for o in gc.get_objects() if isinstance(o, Node)])}")
    
    # Удаляем локальные ссылки
    del node_a, node_b
    
    print(f"Объектов в памяти после удаления ссылок: {len([o for o in gc.get_objects() if isinstance(o, Node)])}")
    print("Объекты не удалены из-за циклических ссылок")
    
    # Запускаем сборщик мусора вручную
    print("\nЗапуск сборщика мусора...")
    collected = gc.collect()
    print(f"Собрано объектов: {collected}")
    print(f"Объектов в памяти после gc: {len([o for o in gc.get_objects() if isinstance(o, Node)])}")
    # Включаем автоматический сборщик мусора обратно
    gc.enable()


demonstrate_garbage_collection()


### Копирование объектов

**Наивное копирование объектов**

In [ ]:
original_number = 10
naive_copied_number = original_number

original_number += 1

print(original_number, naive_copied_number) # изменилось только оригинальное число

In [ ]:
original_list = [1, 2, 3]
naive_copied_list = original_list

original_list.append(4)

print(original_list, naive_copied_list) # оба списка изменились!

Как мы видим, если (для списков) мы присваиваем переменной значение другой переменной, то они ссылаются на один и тот же объект.

Однако, мы можем вручную скопировать список через цикл или через list comprehension.

Также можно использовать метод `copy` из модуля `copy`.

In [ ]:
from copy import copy

def _loop_copy(lst:list) -> list:
    copied = []
    for elem in lst:
        copied.append(elem)
    return copied

def _list_comp_copy(lst:list) -> list:
    return [elem for elem in lst]


original_list = [1, 2, 3]

loop_copied = _loop_copy(original_list)
list_comp_copied = _list_comp_copy(original_list)
copy_copied = copy(original_list)

original_list.append(4)

print(original_list)
print(loop_copied)
print(list_comp_copied)
print(copy_copied)

Однако, со списками списков всё сложнее. 

In [ ]:
list_of_lists = [
    [0, 1],
    [3, 4]
]

naive_copy = list_of_lists
loop_copy = _loop_copy(list_of_lists)
list_comp_copy = _list_comp_copy(list_of_lists)
copy_copy = copy(list_of_lists)

list_of_lists.append([5, 6]) # Изменится только у naive_copy,
                             # потому что скопирована ссылка на тот же объект

list_of_lists[0].append(2)   # А вот тут 2 добавится ко всем объектам, потому что мы скопировали
                             # только внешний список, а внутренние отсылают к оригинальным объектам
print(list_of_lists)
print(naive_copy)
print(loop_copy)
print(list_comp_copy)
print(copy_copy)

Чтобы скопировать сложный объект (список списков или словарь содержащий списки или словарь словарей), нужно использовать метод `deepcopy` модуля `copy`. Он рекурсивно копирует все объекты вложенные в исходный объект.

In [ ]:
from copy import deepcopy

list_of_lists = [
    [0, 1],
    [3, 4]
]

deep_copy = deepcopy(list_of_lists)

list_of_lists.append([5, 6])
list_of_lists[0].append(2)   
                             
print(list_of_lists)
print(deep_copy)

### Переиспользование объектов

По сути -- повторение с прошлой лекции. Python использует таблицу ссылок, чтобы не создавать новые объекты, а переиспользовать уже существующие для небольших чисел и строк.

In [ ]:
def demonstrate_object_reuse():
    """Демонстрирует переиспользование объектов Python."""
    
    # Маленькие целые числа переиспользуются
    a = 5
    b = 5
    print(f"a is b для 5: {a is b}")  # True
    
    # Строки тоже могут переиспользоваться
    s1 = "hello"
    s2 = "hello"
    print(f"s1 is s2 для 'hello': {s1 is s2}")  # True
    
    # Но не всегда!
    s3 = "hello world" * 1000
    s4 = "hello world" * 1000
    print(f"s3 is s4 для длинных строк: {s3 is s4}")  # Может быть False

    slovo = "slovo"
    slo = "slo"
    vo = "vo"
    also_slovo = slo + vo
    print(slovo, also_slovo)
    print(slovo == also_slovo, slovo is also_slovo)


demonstrate_object_reuse()

## Семинар 2.

### Задание 1. 

Напишите собственную функцию `my_deepcopy`. Известно, что на вход подается список списков (любой глубины вложенности). Можно использовать `copy.copy` в реализации. `copy.deepcopy` не использовать.

In [ ]:
from copy import copy, deepcopy

def my_deepcopy(lst:list) -> list:
    # Ваш код
    return ...

list_of_lists = [
    ['a', 'b'],
    ['c'],
    [1, [2, [3, 4, 5]]]
]

deep_copy = deepcopy(list_of_lists)
my_deep_copy = my_deepcopy(list_of_lists)

list_of_lists.append(["e", "f"])
list_of_lists[1].append("d")
list_of_lists[2][1][1].append("Hello!")

print(deep_copy == my_deep_copy)

### Задание 2.

Изучите функцию, приведённую ниже. Что в ней не так? В чём некорректность? Что нужно исправить? Как?

In [ ]:

def magical_append(item, lst:list = []) -> list:
    lst.append(item)
    return lst
